在序列学习中，我们以往假设的目标是： 在给定观测的情况下 （例如，在时间序列的上下文中或在语言模型的上下文中）， 对下一个输出进行建模。 虽然这是一个典型情景，但不是唯一的。 还可能发生什么其它的情况呢？ 我们考虑以下三个在文本序列中填空的任务：

+ 我___。
+ 我___饿了。
+ 我___饿了，我可以吃半头猪。

根据可获得的信息量，我们可以用不同的词填空， 如“很高兴”（“happy”）、“不”（“not”）和“非常”（“very”）。 很明显，每个短语的“下文”传达了重要信息（如果有的话）， 而这些信息关乎到选择哪个词来填空， 所以无法利用这一点的序列模型将在相关任务上表现不佳。

### 9.4.2. 双向模型
如果我们希望在循环神经网络中拥有一种机制， 使之能够提供与隐马尔可夫模型类似的前瞻能力， 我们就需要修改循环神经网络的设计。 幸运的是，这在概念上很容易， 只需要增加一个“从最后一个词元开始从后向前运行”的循环神经网络， 而不是只有一个在前向模式下“从第一个词元开始运行”的循环神经网络。 双向循环神经网络（bidirectional RNNs） 添加了反向传递信息的隐藏层，以便更灵活地处理此类信息。

![双向循环神经网络架构](imgs/9_4_2双向模型1.png)

#### 9.4.2.1. 定义
![双向模型定义](imgs/9_4_2双向模型1.png)

#### 9.4.2.2. 模型的计算代价及其应用
双向循环神经网络的一个关键特性是：使用来自序列两端的信息来估计输出。

也就是说，我们使用来自过去和未来的观测信息来预测当前的观测。 但是在对下一个词元进行预测的情况中，这样的模型并不是我们所需的。 因为在预测下一个词元时，我们终究无法知道下一个词元的下文是什么， 所以将不会得到很好的精度。

具体地说，在训练期间，我们能够利用过去和未来的数据来估计现在空缺的词； 而在测试期间，我们只有过去的数据，因此精度将会很差。 下面的实验将说明这一点。

另一个严重问题是，双向循环神经网络的计算速度非常慢。
##### 其主要原因是网络的前向传播需要在双向层中进行前向和后向递归， 并且网络的反向传播还依赖于前向传播的结果。 因此，梯度求解将有一个非常长的链。

双向层的使用在实践中非常少，并且仅仅应用于部分场合。 例如，填充缺失的单词、词元注释（例如，用于命名实体识别） 以及作为序列处理流水线中的一个步骤对序列进行编码（例如，用于机器翻译）。

### 9.4.3. 双向循环神经网络的错误应用
由于双向循环神经网络使用了过去的和未来的数据， 所以我们不能盲目地将这一语言模型应用于任何预测任务。 尽管模型产出的困惑度是合理的， 该模型预测未来词元的能力却可能存在严重缺陷。 我们用下面的示例代码引以为戒，以防在错误的环境中使用它们。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

# 加载数据
batch_size, num_steps, device = 32, 35, d2l.try_gpu()
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
# 通过设置“bidirective=True”来定义双向LSTM模型
vocab_size, num_hiddens, num_layers = len(vocab), 256, 2
num_inputs = vocab_size
lstm_layer = nn.LSTM(num_inputs, num_hiddens, num_layers, bidirectional=True)
model = d2l.RNNModel(lstm_layer, len(vocab))
model = model.to(device)
# 训练模型
num_epochs, lr = 500, 1
d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.